# Chapter 3: Better Living Through Better Hashing

## Imports and Setup

In [1]:
from datetime import date
import calendar
from matplotlib import pyplot as plt

## Associting Values with Keys

In [2]:
month_length = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
key_array = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

idx = key_array.index("February")
print(f"Febrary has {month_length[idx]} days")

Febrary has 28 days


In [3]:
def print_month(month, year):
    idx = key_array.index(month)
    day = 1

    wd = date(year, idx + 1, day).weekday()
    wd = (wd + 1) % 7
    end = month_length[idx]
    if calendar.isleap(year) and idx == 1:
        end += 1
    print(f"{month} {year}".center(20))
    print("Su Mo Tu We Th Fr Sa")
    print("   " * wd, end="")
    while day <= end:
        print(f"{day:2d} ", end="")
        wd = (wd + 1) % 7
        day += 1
        if wd == 0:
            print()
    print()

In [4]:
print_month("February", 2024)

   February 2024    
Su Mo Tu We Th Fr Sa
             1  2  3 
 4  5  6  7  8  9 10 
11 12 13 14 15 16 17 
18 19 20 21 22 23 24 
25 26 27 28 29 


In [5]:
days_in_month = {
    "January": 31,
    "February": 28,
    "March": 31,
    "April": 30,
    "May": 31,
    "June": 30,
    "July": 31,
    "August": 31,
    "September": 30,
    "October": 31,
    "November": 30,
    "December": 31,
}
print(f"April has {days_in_month['April']} days")

April has 30 days


In [6]:
# Convert a word to a base 26 integer
def base26(word: str) -> int:
    val = 0
    for ch in word.lower():
        new_digit = ord(ch) - ord("a")
        val = 26 * val + new_digit
    return val


base26("Marlon")

142883117

## A Hashtable Structure for (Key, Value) Pairs

In [7]:
class Entry:
    def __init__(self, key, value):
        self.key = key
        self.value = value

In [8]:
# Ineffective Hastable implementation
class Hashtable:
    def __init__(self, M: int = 10):
        self.table = [None] * M
        self.M = M

    def get(self, key: int) -> str:
        hc = hash(key) % self.M
        return self.table[hc].value if self.table[hc] else None

    def put(self, key: int, value: str):
        hc = hash(key) % self.M
        entry = self.table[hc]
        if entry:
            if entry.key == key:
                entry.value = value
            else:
                raise RuntimeError("Key already exists")
        else:
            self.table[hc] = Entry(key, value)


In [9]:
table = Hashtable(1000)
table.put("April", "30")
table.put("May", "31")
table.put("June", "30")

print(table.get("April"))
print(table.get("August"))

30
None


## Detecting and Resolving Collisions with Linear Probing

In [10]:
# Applying Linear Probing
class Hashtable:
    def __init__(self, M: int = 10):
        self.table = [None] * M
        self.M = M
        self.N = 0

    def get(self, key):
        hc = hash(key) % self.M
        while self.table[hc]:
            if self.table[hc].key == key:
                return self.table[hc].valu
            hc = (hc + 1) % self.M
        return None

    def put(self, key, value):
        hc = hash(key) % self.M
        while self.table[hc]:
            if self.table[hc].key == key:
                self.table[hc].value = value
                return
            hc = (hc + 1) % self.M
        if self.N >= self.M - 1:
            raise RuntimeError("Hashtable is full")
        self.table[hc] = Entry(key, value)
        self.N += 1

## Separate Chaining with Linked Lists

In [15]:
class LinkedEntry:
    def __init__(self, key, value, next=None):
        self.key = key
        self.value = value
        self.next = next

In [18]:
class Hashtable:
    def __init__(self, M=10):
        self.table = [None] * M
        self.M = M
        self.N = 0

    def get(self, key):
        hc = hash(key) % self.M
        entry = self.table[hc]
        while entry:
            if entry.key == key:
                return entry.value
            entry = entry.next
        return None

    def put(self, key, value):
        hc = hash(key) % self.M
        entry = self.table[hc]
        while entry:
            if entry.key == key:
                entry.value = value
                return
            entry = entry.next

        self.table[hc] = LinkedEntry(key, value, self.table[hc])
        self.N += 1

    def remove(self, key):
        hc = hash(key) % self.M
        entry = self.table[hc]
        prev = None
        while entry:
            if entry.key == key:
                if prev:
                    prev.next = entry.next
                else:
                    self.table[hc] = entry.next
                self.N -= 1
                return entry.value
            prev = entry
            entry = entry.next
        return None

## Growing Hastables

In [20]:
class DynamicHashtable:
    def __init__(self, M=10):
        self.table = [None] * M
        self.M = M
        self.N = 0
        self.load_factor = 0.75
        self.threshold = min(M * self.load_factor, M - 1)

    def put(self, key, value):
        hc = hash(key) % self.M
        entry = self.table[hc]
        while entry:
            if entry.key == key:
                entry.value = value
                return
            entry = entry.next
        self.table[hc] = LinkedEntry(key, value, self.table[hc])
        self.N += 1
        if self.N > self.threshold:
            self.resize(2 * self.M + 1)

    def resize(self, new_size):
        temp = DynamicHashtable(new_size)
        for n in self.table:
            while n:
                temp.put(n.key, n.value)
                n = n.next
        self.table = temp.table
        self.M = temp.M
        self.threshold = self.load_factor * self.M


## Perfect Hashing

In [21]:
G = [0, 8, 1, 4, 7, 10, 2, 0, 9, 11, 1, 5]
s1 = [9, 4, 8, 6, 6]
s2 = [2, 10, 6, 3, 5]


def hash_f(key, T):
    return sum(T[i % 5] * ord(c) for i, c in enumerate(key)) % 12


def perfect_hash(key):
    return (G[hash_f(key, s1)] + G[hash_f(key, s2)]) % 12